In [11]:
from databricks import sql
import os
from openpyxl import load_workbook
import pandas as pd
from datetime import date, datetime, timedelta
from win32com import client
import xlwings as xw
import shutil

In [2]:
today = date.today()
CurrentDate = today.strftime("%Y-%m-%d")

In [3]:
with open('APIKEYS.txt', 'r') as f: 
    # Read the contents of the file into a list 
    lines = f.readlines() 
    # Create an empty dictionary 
    APIKEYS = {} 
    # Loop through the list of lines 
    for line in lines: 
        # Split the line into key-value pairs 
        key, value = line.strip().split(',') 
        # Store the key-value pairs in the dictionary 
        APIKEYS[key] = value 
# The dictionary 'data' now contains the contents of the text file 

In [4]:
with open('INT OTIF.txt', 'r') as f: 
    # Read the contents of the file into a list 
    lines = f.readlines() 
    # Create an empty dictionary 
    LastUpdate = {} 
    # Loop through the list of lines 
    for line in lines: 
        # Split the line into key-value pairs 
        key, value = line.strip().split(',') 
        # Store the key-value pairs in the dictionary 
        LastUpdate[key] = value 
# The dictionary 'data' now contains the contents of the text file 

In [5]:

date_obj = datetime.strptime(LastUpdate['LastUpdatedDate'], '%Y-%m-%d')
incremented_date = date_obj + timedelta(days=1)
incremented_date_str = incremented_date.strftime('%Y-%m-%d')

In [7]:
# Get Parcel data from DB and place in table called "Parcel_result"
with sql.connect(server_hostname = APIKEYS['server_hostname'],
                 http_path       = APIKEYS['http_path'],
                 access_token    = APIKEYS['access_token']) as connection:

  with connection.cursor() as cursor:
    cursor.execute(f"select * from WhseFin_SharedArea.Results_Int_OTIF_ParcelDetail where Date(FileDateTime) between '{incremented_date_str}' and '{CurrentDate}' order by Generation_Date desc")
    Parcel_result = cursor.fetchall()

#Get OTIF results from DB and place in table called "OTIF_result"
with sql.connect(server_hostname = APIKEYS['server_hostname'],
                 http_path       = APIKEYS['http_path'],
                 access_token    = APIKEYS['access_token']) as connection:

  with connection.cursor() as cursor:
    cursor.execute(f"""Select 

Generation_Date,
Whse,
MOD,
Depot,
Destination,
Target_Despatch_Date,
Target_Despatch_Time as Target_Despatch_Time,
OTIF,
Parcels,
No_Record_Of_Despatch_Or_Pack_Items,
Item_Cancelled,
No_Stocked_Items,
Item_Despatched_On_Time,
Item_Despatched_Late,
Total_Items,
FinweekNo,
case when OTIF = 'On Time in Full'	then 'Success'
when OTIF = 'Parcel Despatched Late' then 'Fail'
when OTIF = 'Parcel Incomplete No Stocked Item - Despatched Late' then 'Fail'
when OTIF = 'Parcel Incomplete No Stocked Item - Despatched On Time'	then 'Success'
when OTIF = 'CHECK'	then 'Fail'
when OTIF = 'Full Parcel No Stock'	then 'Fail' end as Category




from WhseFin_SharedArea.Results_Int_OTIF as o
left join parcel_tracking.vw_d_date as d  on o.Target_Despatch_Date =d.date
Where Date(FileDateTime) between '{incremented_date_str}' and '{CurrentDate}'


order by Target_Despatch_Date desc""")
    OTIF_result = cursor.fetchall()



In [16]:

#convert tables to pandas, listing all columns
Parcel_data = pd.DataFrame(Parcel_result, columns = ['Generation_Date',	'Parcel_Barcode',	'account_no',	'order_date',	'promise_date',	'Whse',	'MOD',	'Batch',	'NoRecord',	'ItemCancelled',	'NoStocked',	'ItemDespatchedOnTime',	'ItemDespatchedLate',	'Total_Items',	'depot',	'Destination',	'packing_station',	'parcel_collate_status',	'grand_collation_flag',	'target_despatch_date',	'target_despatch_time',	'despatch_date',	'despatch_time',	'latest_status_description',	'latest_status_date',	'latest_status_time',	'delivered_date',	'delivered_time',	'delivered_status',	'delivered_status_code',	'delivered_sub_status_code',	'OTIF',	'FileDateTime',	'FileDateYear',	'FileDateDay',	'FileDateMonth'])
OTIF_result = pd.DataFrame(OTIF_result, columns = ['Generation_Date',	'Whse',	'MOD',	'Depot',	'Destination',	'Target_Despatch_Date',	'Target_Despatch_Time',	'OTIF',	'Parcels',	'No_Record_Of_Despatch_Or_Pack_Items',	'Item_Cancelled',	'No_Stocked_Items',	'Item_Despatched_On_Time',	'Item_Despatched_Late',	'Total_Items','FinweekNo','Category'])

In [18]:
#get current week
curr_week = OTIF_result.iloc[0][-2].astype('str')
print(curr_week)
#Get current season
def season():
    if OTIF_result.iloc[0][-2].astype('int32') < 27:
        return 'SS' + Parcel_data.iloc[0][-3].astype('str')[2:4]
    else:
        return 'AW' + Parcel_data.iloc[0][-3].astype('str')[2:4]

curr_season = season()

15


In [ ]:
curr_season

'SS23'

In [ ]:

#any datetime columns need to be turned to date
Parcel_data['FileDateTime'] = pd.to_datetime(Parcel_data['FileDateTime']).dt.date

In [ ]:
#Get current date to save files with a dated name
today2 = date.today()
date2 = today.strftime("%d.%m.%Y")

In [ ]:
#create and save the parcel data files
Parcel_data.to_excel('C:\\Users\\CFPRICE\\Desktop\\OTIFS\\INT\\{0} - Int.xlsx'.format(date2), sheet_name='Data',index = False)

In [ ]:

#create app to make updating excel a background process
app = xw.App(visible = False)

#Select File and sheet to update
wb = app.books.open(f"C:\\Users\\CFPRICE\\Desktop\\OTIFS\\INT\\International OTIF - Week {LastUpdate['LastFileWeek']}.xlsx")
ws = wb.sheets["Summary Data"]

#find which row we need to place data
last_row = ws.range(1,1).end('down').row
Import_row = last_row + 1

In [ ]:
#Place the results Data
ws.range("A{0}".format(Import_row)).options(index=False).value = OTIF_result

In [ ]:
#save the file
wb.save(f'C:\\Users\\CFPRICE\\Desktop\\OTIFS\\INT\\International OTIF - Week {curr_week}.xlsx')

#stop the app
app.kill()

In [ ]:
#In or out of office 
def OfficeCheck():
    if os.path.exists('T:\\Common\\Management Information') == False:
        return 'Home'
    elif os.path.exists('T:\\Common\\Management Information') == True:
        return 'Office'

In [ ]:
OfficeFile = 'T:\\Common\\Management Information\\On Time in Full\\International\\{season}\\Week {week}\\International OTIF - Week {week}.xlsx'
DesktopLoc = 'C:\\Users\\CFPRICE\\Desktop\\OTIFS\\INT\\International OTIF - Week {week}.xlsx'
OfficeLoc =  'T:\\Common\\Management Information\\On Time in Full\\International\\{season}\\Week {week}'

In [ ]:
def Save(DesktopLoc,OfficeLoc,OfficeFile,LastUpdate):
    #If at home, update as normal
    if OfficeCheck() == 'Home':
        return

    #If in Office and the last update was also in the office, save the latest copy in MI
    elif OfficeCheck() == 'Office' and LastUpdate['LastUpdatedDate'] == LastUpdate['LastUpdatedOfficeDate']:

        #Create file path if not exists for recent update
        if not os.path.exists(OfficeLoc.format(season = curr_season,week = curr_week)):
            os.mkdir(OfficeLoc.format(season = curr_season,week = curr_week))
            
        #copy latest file to MI from desktop
        shutil.copyfile(DesktopLoc.format(week = curr_week),OfficeFile.format(season = curr_season, week = curr_week))
    
    #If in office and last update was at home
    elif OfficeCheck() == 'Office' and LastUpdate['LastUpdatedDate'] == LastUpdate['LastUpdatedHomeDate']:

        #Still the same week so just need to check file path exists and save file
        if LastUpdate['LastFileHomeWeek'] == LastUpdate['LastFileWeek']:
            if not os.path.exists(OfficeLoc.format(season = curr_season,week = curr_week)):
                os.mkdir(OfficeLoc.format(season = curr_season, week = curr_week))
            shutil.copyfile(DesktopLoc.format(week = curr_week),OfficeFile.format(season = curr_season, week = curr_week))

        #If we are not in the same week, we need to save last weeks file down and this weeks file into the correct paths
        elif LastUpdate['LastFileWeek'] != LastUpdate['LastFileHomeWeek']:

            #save last weeks file
            if not os.path.exists(OfficeLoc.format(season = LastUpdate['LastUpdatedHomeSeason'], week = LastUpdate['LastFileHomeWeek'])):
                os.mkdir(OfficeLoc.format(season = LastUpdate['LastUpdatedHomeSeason'], week = LastUpdate['LastFileHomeWeek']))
            shutil.copyfile(DesktopLoc.format(week = LastUpdate['LastFileHomeWeek']),OfficeFile.format(season = LastUpdate['LastUpdatedHomeSeason'], week = LastUpdate['LastFileHomeWeek']))

            #save this weeks file
            if not os.path.exists(OfficeLoc.format(season = curr_season,week = curr_week)):
                os.mkdir(OfficeLoc.format(season = curr_season, week = curr_week))
            shutil.copyfile(DesktopLoc.format(week = curr_week),OfficeFile.format(season = curr_season, week = curr_week))

            #remove last weeks file from desktop to save space
            os.remove(DesktopLoc.format(week = LastUpdate['LastFileWeek']))
Save(DesktopLoc,OfficeLoc,OfficeFile,LastUpdate)


In [ ]:
# Read the contents of the file into memory
with open('C:\\Users\\CFPRICE\\Desktop\\OTIFS\\INT\\INT OTIF.txt', 'r') as f:
    lines = f.readlines()

# Update the values in memory
for i, line in enumerate(lines):
    key, value = line.strip().split(',')

    if key == 'LastFileWeek':
        value = str(curr_week)

    if key == 'LastUpdatedDate':
        value = str(CurrentDate)# update the values

    if key == 'LastUpdatedLoc':
        value = OfficeCheck()

    if key == 'LastUpdatedSeason':
        value = curr_season

    if key == f'LastUpdated{OfficeCheck()}Date':
        value = str(CurrentDate)

    if key == f'LastFile{OfficeCheck()}Week':
        value = str(curr_week)

    if key == f'LastUpdated{OfficeCheck()}Season':
        value = curr_season
        
    lines[i] = key + ',' + value + '\n'

with open('C:\\Users\\CFPRICE\\Desktop\\OTIFS\\INT\\INT OTIF.txt', 'w') as f:
    f.writelines(lines)